In [2]:
import os
os.listdir()

['.python-version',
 '.venv',
 'dataset',
 'main.ipynb',
 'main.py',
 'pyproject.toml',
 'README.md',
 'uv.lock']

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("dataset/description.txt")
df

,Train data:
0,ID ::: TITLE ::: GENRE ::: DESCRIPTION
1,ID ::: TITLE ::: GENRE ::: DESCRIPTION
2,ID ::: TITLE ::: GENRE ::: DESCRIPTION
3,ID ::: TITLE ::: GENRE ::: DESCRIPTION
4,Test data:
5,ID ::: TITLE ::: DESCRIPTION
6,ID ::: TITLE ::: DESCRIPTION
7,ID ::: TITLE ::: DESCRIPTION
8,ID ::: TITLE ::: DESCRIPTION
9,Source:


In [8]:
def load_data(file_path):
    with open(file_path, "r",encoding="utf-8") as f:
        data = f.readlines()
    data = [line.strip().split(':::') for line in data]    
    return data




In [27]:
train_data = load_data("dataset/train_data.txt")
train_df = pd.DataFrame(train_data, columns=["movie_id", "title", "genre", "description"])

test_data = load_data("dataset/test_data.txt")
test_df = pd.DataFrame(test_data, columns=["movie_id", "title", "description"])

test_data_solution = load_data("dataset/test_data_solution.txt")
test_data_solution_df = pd.DataFrame(test_data_solution, columns=["movie_id", "title", "genre", "description"])


In [24]:
train_df.isnull().sum() # check for Nan Values

movie_id       0
title          0
genre          0
description    0
dtype: int64

In [25]:
test_df.head()

,movie_id,title,description
0,1,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apar..."
1,2,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty ch..."
2,3,Off the Beaten Track (2010),One year in the life of Albin and his family ...
3,4,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with hi..."
4,5,Er nu zhai (1955),Before he was known internationally as a mart...


In [26]:
train_df.head()

,movie_id,title,genre,description
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [28]:

test_data_solution_df.head()

,movie_id,title,genre,description
0,1,Edgar's Lunch (1998),thriller,"L.R. Brane loves his life - his car, his apar..."
1,2,La guerra de papá (1977),comedy,"Spain, March 1964: Quico is a very naughty ch..."
2,3,Off the Beaten Track (2010),documentary,One year in the life of Albin and his family ...
3,4,Meu Amigo Hindu (2015),drama,"His father has died, he hasn't spoken with hi..."
4,5,Er nu zhai (1955),drama,Before he was known internationally as a mart...


feature extracting: TF-IDF

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=10000)

X_train_tfidf = vectorizer.fit_transform(train_df["description"])
X_test_tfidf = vectorizer.transform(test_df["description"])


print("X_train_tfidf.shape",X_train_tfidf.shape)
print("X_test_tfidf.shape",X_test_tfidf.shape)



X_train_tfidf.shape (54214, 10000)
X_test_tfidf.shape (54200, 10000)


label encoding

In [31]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df["genre"])
print(f"unique genres in train data: {label_encoder.classes_}")



unique genres in train data: [' action ' ' adult ' ' adventure ' ' animation ' ' biography ' ' comedy '
 ' crime ' ' documentary ' ' drama ' ' family ' ' fantasy ' ' game-show '
 ' history ' ' horror ' ' music ' ' musical ' ' mystery ' ' news '
 ' reality-tv ' ' romance ' ' sci-fi ' ' short ' ' sport ' ' talk-show '
 ' thriller ' ' war ' ' western ']


Logistic Regression Algo

In [32]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(max_iter=200)

In [33]:
lr_model.fit(X_train_tfidf,y_train)

LogisticRegression(max_iter=200)

In [34]:
y_pred = lr_model.predict(X_test_tfidf)
predictied_genres = label_encoder.inverse_transform(y_pred)


test_df["predicted_genre"] = predictied_genres
test_df[['title','predicted_genre']].head(10)




,title,predicted_genre
0,Edgar's Lunch (1998),drama
1,La guerra de papá (1977),drama
2,Off the Beaten Track (2010),documentary
3,Meu Amigo Hindu (2015),drama
4,Er nu zhai (1955),drama
5,Riddle Room (2016),drama
6,L'amica (1969),drama
7,Ina Mina Dika (1989),comedy
8,Equinox Special: Britain's Tornados (2005),documentary
9,Press (2011),drama


In [38]:
merger_df = pd.merge(test_data_solution_df[['movie_id','genre']],test_df[['movie_id','predicted_genre']], on='movie_id')
merger_df.head(5)


,movie_id,genre,predicted_genre
0,1,thriller,drama
1,2,comedy,drama
2,3,documentary,documentary
3,4,drama,drama
4,5,drama,drama


In [41]:
from sklearn.metrics import accuracy_score
ac = accuracy_score(merger_df['genre'],merger_df['predicted_genre'])
ac

0.5945387453874539

Model Evaluation 